In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.baseline_functions as base

# restore saved variables
%store -r summary_M6_KY

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2


In [2]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv")
data = data.drop(['fta_risk_score_raw','nca_risk_score_raw','pvf_risk_score_raw'], axis=1)
x = data.loc[:,:'current_violence20']
y = data['misdemeanor_six_month'].values

In [3]:
#### Logistic
c = [1e-4, 1e-3, 0.01, 0.1, 1]
logistic_summary = base.Logistic(X=x,
                                 Y=y,
                                 C=c,
                                 seed=816)
#### Lasso
c = [1e-4, 1e-3, 0.01, 0.1, 1]
lasso_summary = base.Lasso(X=x,
                           Y=y,
                           C=c,
                           seed=816)

#### LinearSVM
c = [1e-4, 1e-3, 0.01, 0.1, 1]
svm_summary = base.LinearSVM(X=x,
                             Y=y, 
                             C=c,
                             seed=816)

#### Random Forest
n_estimators =  [100,150,200]
depth = [6,7,8]
rf_summary = base.RF(X=x,
                     Y=y, 
                     depth=depth, 
                     estimators=n_estimators, 
                     seed=816)

#### XGBoost
learning_rate = [0.1]
depth = [3,4]
n_estimators = [100,150]
xgb_summary = base.XGB(X=x,
                       Y=y, 
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators, 
                       seed=816)

#### save results
summary_M6_KY = {"Logistic": logistic_summary,
                 "Lasso": lasso_summary,
                 "LinearSVM": svm_summary,
                 "RF": rf_summary,
                 "XGBoost": xgb_summary}
%store summary_M6_KY

model_id, score_thresholds 0 {'rank_abs': [4674]}
get_disparity_predefined_group()


C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:87: RuntimeWarning: invalid value encountered in longlong_scalars
  fnr = fn / (fn + tp)


model_id, score_thresholds 0 {'rank_abs': [4664]}
get_disparity_predefined_group()


C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:83: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv = tp / (tp + fp)
C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:96: RuntimeWarning: divide by zero encountered in longlong_scalars
  ratio = fn / fp


model_id, score_thresholds 0 {'rank_abs': [4863]}
get_disparity_predefined_group()
model_id, score_thresholds 0 {'rank_abs': [4779]}
get_disparity_predefined_group()


...

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:83: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv = tp / (tp + fp)
C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:96: RuntimeWarning: divide by zero encountered in longlong_scalars
  ratio = fn / fp


model_id, score_thresholds 0 {'rank_abs': [25]}
get_disparity_predefined_group()


C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:83: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv = tp / (tp + fp)
C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:87: RuntimeWarning: invalid value encountered in longlong_scalars
  fnr = fn / (fn + tp)
C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:96: RuntimeWarning: invalid value encountered in longlong_scalars
  ratio = fn / fp
C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:96: RuntimeWarning: divide by zero encountered in longlong_scalars
  ratio = fn / fp


model_id, score_thresholds 0 {'rank_abs': [17]}
get_disparity_predefined_group()
Stored 'summary_M6_KY' (dict)


C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:83: RuntimeWarning: invalid value encountered in longlong_scalars
  ppv = tp / (tp + fp)
C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:96: RuntimeWarning: divide by zero encountered in longlong_scalars
  ratio = fn / fp
C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:87: RuntimeWarning: invalid value encountered in longlong_scalars
  fnr = fn / (fn + tp)
C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:96: RuntimeWarning: invalid value encountered in longlong_scalars
  ratio = fn / fp


In [4]:
results = []
auc = [] ## for table writing -- the performance range 
for model_name, model_summary in summary_M6_KY.items():
    results.append([model_name, np.mean(model_summary['holdout_test_auc']), np.mean(model_summary['auc_diffs'])])
    auc.append(np.mean(model_summary['holdout_test_auc']))
results

[['Logistic', 0.7190538077486869, 0.0026642769364531027],
 ['Lasso', 0.7191346619548419, 0.00280124529715049],
 ['LinearSVM', 0.7216988185441974, 0.002130797352633018],
 ['RF', 0.728112799114691, 0.02025640513694067],
 ['XGBoost', 0.741817764139358, 0.020304257389423253]]

In [5]:
params = [[model_name, model_summary['best_param']] for model_name, model_summary in summary_M6_KY.items()]
params

[['Logistic',
  [{'C': 0.01}, {'C': 0.1}, {'C': 0.01}, {'C': 0.01}, {'C': 0.01}]],
 ['Lasso', [{'C': 0.1}, {'C': 0.1}, {'C': 0.1}, {'C': 0.1}, {'C': 0.1}]],
 ['LinearSVM', [{'C': 0.1}, {'C': 1}, {'C': 0.01}, {'C': 1}, {'C': 0.1}]],
 ['RF',
  [{'max_depth': 7, 'n_estimators': 150},
   {'max_depth': 7, 'n_estimators': 150},
   {'max_depth': 7, 'n_estimators': 100},
   {'max_depth': 7, 'n_estimators': 50},
   {'max_depth': 7, 'n_estimators': 150}]],
 ['XGBoost',
  [{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100},
   {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100},
   {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100},
   {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100},
   {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}]]]

In [7]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\Baselines\\Six Month\\"
results = [["Misdemeanor", np.str((round(np.mean(logistic_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(logistic_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(lasso_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(lasso_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(svm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(svm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(rf_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(rf_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(xgb_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(xgb_summary['holdout_test_auc']), 3)) + ")", 
            round(np.max(auc) - np.min(auc), 3)]]
with open(path + 'Six Month Baseline Summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)